In [2]:
import pandas as pd 
import numpy as np
import json
from collections import defaultdict
import copy 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt

In [3]:
masterData = pd.read_csv('dataMachine/TFT_master_MatchData.csv')

In [4]:
masterData.head()

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion
0,KR_4242528808,2233.449951,8,37,1,2225.243408,"{'DarkStar': 2, 'Protector': 4, 'Rebel': 1, 'S...","{'JarvanIV': {'items': [12], 'star': 3}, 'Sona..."
1,KR_4242528808,2233.449951,9,37,2,2225.243408,"{'Blaster': 2, 'Chrono': 1, 'Cybernetic': 1, '...","{'Malphite': {'items': [], 'star': 2}, 'Blitzc..."
2,KR_4242528808,2233.449951,8,35,3,2138.763428,"{'Blaster': 3, 'Chrono': 2, 'Cybernetic': 1, '...","{'Malphite': {'items': [], 'star': 2}, 'Blitzc..."
3,KR_4242528808,2233.449951,8,33,4,2003.865967,"{'Chrono': 2, 'Cybernetic': 2, 'DarkStar': 2, ...","{'Leona': {'items': [], 'star': 2}, 'Mordekais..."
4,KR_4242528808,2233.449951,7,31,5,1885.395386,"{'Blaster': 1, 'DarkStar': 1, 'Demolitionist':...","{'KhaZix': {'items': [], 'star': 2}, 'KaiSa': ..."


In [6]:
masterData.shape

(79999, 8)

In [7]:
masterData = masterData[:10000]

In [8]:
masterData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gameId          10000 non-null  object 
 1   gameDuration    10000 non-null  float64
 2   level           10000 non-null  int64  
 3   lastRound       10000 non-null  int64  
 4   Ranked          10000 non-null  int64  
 5   ingameDuration  10000 non-null  float64
 6   combination     10000 non-null  object 
 7   champion        10000 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 625.1+ KB


In [10]:
masterData.describe()

,gameDuration,level,lastRound,Ranked,ingameDuration
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,2148.575186,7.955200,33.572700,4.494400,1894.087427
std,150.441133,0.720031,4.370884,2.296279,250.242402
min,181.108841,3.000000,5.000000,0.000000,178.951233
25%,2054.361328,8.000000,31.000000,2.000000,1727.061890
50%,2126.780884,8.000000,34.000000,4.000000,1909.463257
75%,2236.780762,8.000000,37.000000,6.000000,2061.805054
max,2673.695312,9.000000,47.000000,8.000000,2665.456299


In [362]:
masterData.describe(include = 'object').T

,count,unique,top,freq
gameId,10000,1250,KR_4283419459,8
combination,10000,4037,"{'Chrono': 2, 'Cybernetic': 1, 'Infiltrator': ...",208
champion,10000,9980,{},21


In [17]:
champion_col = []
for d in masterData.champion:
    col = list(json.loads(d.replace("'", "\"")).values())
    champion_col.append(col)

In [225]:
n = []
inven = []
for i in range (len(champion_col)): 
    #print(champion_col2[i])
    data = 0
    it = []
    for j in range (len(champion_col[i])):
        #print(i+1,"번유저의 ",j,'번째 챔피언이 가지고 있는 아이템:', champion_col[i][j].get('items'))
        data += (len((champion_col[i][j].get('items'))))
        it.append((champion_col[i][j].get('items')))
    n.append(data)
    inven.append(it)
#print(n)

#champion_col2[0][1].get('items')

In [226]:
for i in range(len(inven)):
    for j in range(len(inven[i])):
        if inven[i][j] == []:
            inven[i][j] = [0]

In [329]:
inven

[[[12], [46], [37, 45], [25, 66, 13], [0], [0], [0], [99, 47, 55]],
 [[0], [0], [0], [56, 13, 29], [0], [36, 6, 27], [0], [37, 34], [17]],
 [[0], [0], [0], [36], [13, 57, 69], [0], [33, 69], [48]],
 [[0], [25, 66], [0], [14, 3], [0], [11, 23, 29], [0], [57]],
 [[0], [44, 36], [0], [16, 15, 19], [67, 55], [0], [0]],
 [[0], [0], [0], [67, 27, 8], [0], [0], [0], [25], [39, 18, 44]],
 [[19, 69], [0], [55, 66], [49], [4], [0], [0]],
 [[0], [2], [2], [5], [0], [44, 15, 49]],
 [[0],
  [0],
  [0],
  [99, 79, 59],
  [12, 12],
  [99, 24, 23],
  [3, 67, 25],
  [37, 34, 36]],
 [[59], [59], [36], [67], [12, 57, 12], [0], [0], [47, 69], [0]],
 [[0], [0], [0], [57, 77], [57, 58, 78], [0], [33, 25, 36], [46, 78]],
 [[0], [0], [57], [99, 14, 15], [2], [0], [37, 15, 44], [38, 66, 45], [29]],
 [[0],
  [0],
  [67],
  [26, 38, 27],
  [0],
  [0],
  [99, 19, 22],
  [39, 15, 44],
  [44, 22]],
 [[67], [0], [0], [0], [12, 57], [33], [67, 5, 15], [99, 14, 15]],
 [[0], [0], [0], [5], [26, 29, 19], [0], [0], [44, 

In [100]:
champion = []
for d in masterData.champion:
    col = list(json.loads(d.replace("'", "\"")).keys())
    champion.append(col)

In [101]:
champion

[['JarvanIV', 'Sona', 'Rakan', 'XinZhao', 'Neeko', 'Karma', 'Soraka', 'Lulu'],
 ['Malphite',
  'Blitzcrank',
  'Vi',
  'Jinx',
  'Soraka',
  'ChoGath',
  'Kayle',
  'MissFortune',
  'Lulu'],
 ['Malphite',
  'Blitzcrank',
  'Ezreal',
  'Vi',
  'Jinx',
  'ChoGath',
  'MissFortune',
  'AurelionSol'],
 ['Leona', 'Mordekaiser', 'Jayce', 'Ashe', 'WuKong', 'Jhin', 'Thresh', 'Ekko'],
 ['KhaZix', 'KaiSa', 'Annie', 'Shaco', 'Rumble', 'Fizz', 'MissFortune'],
 ['Ziggs',
  'Yasuo',
  'Sona',
  'MasterYi',
  'Jinx',
  'Kayle',
  'MissFortune',
  'Gangplank',
  'AurelionSol'],
 ['Caitlyn', 'Leona', 'Mordekaiser', 'Jayce', 'Karma', 'WuKong', 'Jhin'],
 ['Graves', 'Leona', 'Lucian', 'Vi', 'Jinx', 'Irelia'],
 ['Ziggs',
  'Malphite',
  'Sona',
  'MasterYi',
  'Jinx',
  'MissFortune',
  'Gangplank',
  'AurelionSol'],
 ['Malphite',
  'Blitzcrank',
  'Vi',
  'Karma',
  'Jinx',
  'ChoGath',
  'MissFortune',
  'Gangplank',
  'Lulu'],
 ['Yasuo',
  'Sona',
  'MasterYi',
  'Jinx',
  'Kayle',
  'MissFortune',
  'A

In [64]:
star = []
for i in range (len(champion_col)): 
    #print(champion_col2[i])
    it = []
    #print(champion_col[i])
    for j in range (len(champion_col[i])):
        it.append(str((champion_col[i][j].get('star'))))
    
    star.append(it)

#print(n)

#champion_col2[0][1].get('items')

In [116]:
cham_star = []
for i in range(len(champion)):
    it = []
    for j in range(len(champion[i])):
        #print(star[i][j] + champion[i][j])
        it.append(star[i][j] + champion[i][j])
    cham_star.append(it)
    
print(cham_star)

[['3JarvanIV', '3Sona', '3Rakan', '3XinZhao', '2Neeko', '2Karma', '1Soraka', '2Lulu'], ['2Malphite', '2Blitzcrank', '1Vi', '2Jinx', '2Soraka', '2ChoGath', '2Kayle', '1MissFortune', '2Lulu'], ['2Malphite', '2Blitzcrank', '2Ezreal', '2Vi', '1Jinx', '2ChoGath', '2MissFortune', '2AurelionSol'], ['2Leona', '2Mordekaiser', '2Jayce', '2Ashe', '2WuKong', '2Jhin', '2Thresh', '2Ekko'], ['2KhaZix', '3KaiSa', '3Annie', '2Shaco', '3Rumble', '1Fizz', '1MissFortune'], ['2Ziggs', '2Yasuo', '2Sona', '2MasterYi', '1Jinx', '2Kayle', '1MissFortune', '1Gangplank', '2AurelionSol'], ['2Caitlyn', '2Leona', '1Mordekaiser', '2Jayce', '2Karma', '2WuKong', '1Jhin'], ['1Graves', '1Leona', '2Lucian', '2Vi', '2Jinx', '2Irelia'], ['1Ziggs', '2Malphite', '2Sona', '2MasterYi', '1Jinx', '2MissFortune', '1Gangplank', '2AurelionSol'], ['2Malphite', '2Blitzcrank', '2Vi', '1Karma', '2Jinx', '2ChoGath', '1MissFortune', '2Gangplank', '2Lulu'], ['2Yasuo', '2Sona', '2MasterYi', '1Jinx', '2Kayle', '1MissFortune', '2AurelionSol',

In [95]:
len(cham_star)

10000

In [117]:
game_major = pd.DataFrame(cham_star)

In [337]:
for i in range(11):
    for j in range(10000):
        if (game_major[i][j] is  None):
            game_major[i][j] = 'NotChampion'

In [338]:
game_major

,0,1,2,3,4,5,6,7,8,9,10
0,3JarvanIV,3Sona,3Rakan,3XinZhao,2Neeko,2Karma,1Soraka,2Lulu,NotChampion,NotChampion,NotChampion
1,2Malphite,2Blitzcrank,1Vi,2Jinx,2Soraka,2ChoGath,2Kayle,1MissFortune,2Lulu,NotChampion,NotChampion
2,2Malphite,2Blitzcrank,2Ezreal,2Vi,1Jinx,2ChoGath,2MissFortune,2AurelionSol,NotChampion,NotChampion,NotChampion
3,2Leona,2Mordekaiser,2Jayce,2Ashe,2WuKong,2Jhin,2Thresh,2Ekko,NotChampion,NotChampion,NotChampion
4,2KhaZix,3KaiSa,3Annie,2Shaco,3Rumble,1Fizz,1MissFortune,NotChampion,NotChampion,NotChampion,NotChampion
...,...,...,...,...,...,...,...,...,...,...,...
9995,2Shen,1Rakan,3Ezreal,2Kassadin,3Kayle,2WuKong,1MissFortune,1Thresh,NotChampion,NotChampion,NotChampion
9996,2Poppy,2Zoe,3Ahri,3Neeko,3Syndra,2Lux,2Soraka,1Lulu,NotChampion,NotChampion,NotChampion
9997,1TwistedFate,2Blitzcrank,2Shen,2Ezreal,2Kassadin,2Irelia,2Kayle,2WuKong,1MissFortune,1Thresh,NotChampion
9998,2JarvanIV,2Sona,2Rakan,2XinZhao,2Neeko,2Karma,2Soraka,NotChampion,NotChampion,NotChampion,NotChampion


In [330]:
game_major2 = pd.DataFrame(inven)

In [331]:
game_major2

,0,1,2,3,4,5,6,7,8,9,10
0,[12],[46],"[37, 45]","[25, 66, 13]",[0],[0],[0],"[99, 47, 55]",None,None,None
1,[0],[0],[0],"[56, 13, 29]",[0],"[36, 6, 27]",[0],"[37, 34]",[17],None,None
2,[0],[0],[0],[36],"[13, 57, 69]",[0],"[33, 69]",[48],None,None,None
3,[0],"[25, 66]",[0],"[14, 3]",[0],"[11, 23, 29]",[0],[57],None,None,None
4,[0],"[44, 36]",[0],"[16, 15, 19]","[67, 55]",[0],[0],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
9995,[0],[0],"[99, 19, 22]","[56, 15, 25]","[22, 37, 23]",[0],[18],"[22, 24]",None,None,None
9996,[0],[0],"[78, 33]","[69, 27]","[46, 46, 15]",[0],"[47, 16]",[78],None,None,None
9997,[0],[67],[7],[0],[0],"[28, 16]","[13, 24, 24]",[0],"[57, 24]",[0],None
9998,[0],[0],"[37, 49, 1]","[69, 15]","[89, 35]",[0],"[1, 12]",None,None,None,None


In [316]:
game_major2.fillna('없음')

,0,1,2,3,4,5,6,7,8,9,10
0,[12],[46],"[37, 45]","[25, 66, 13]",[0],[0],[0],"[99, 47, 55]",없음,없음,없음
1,[0],[0],[0],"[56, 13, 29]",[0],"[36, 6, 27]",[0],"[37, 34]",[17],없음,없음
2,[0],[0],[0],[36],"[13, 57, 69]",[0],"[33, 69]",[48],없음,없음,없음
3,[0],"[25, 66]",[0],"[14, 3]",[0],"[11, 23, 29]",[0],[57],없음,없음,없음
4,[0],"[44, 36]",[0],"[16, 15, 19]","[67, 55]",[0],[0],없음,없음,없음,없음
...,...,...,...,...,...,...,...,...,...,...,...
9995,[0],[0],"[99, 19, 22]","[56, 15, 25]","[22, 37, 23]",[0],[18],"[22, 24]",없음,없음,없음
9996,[0],[0],"[78, 33]","[69, 27]","[46, 46, 15]",[0],"[47, 16]",[78],없음,없음,없음
9997,[0],[67],[7],[0],[0],"[28, 16]","[13, 24, 24]",[0],"[57, 24]",[0],없음
9998,[0],[0],"[37, 49, 1]","[69, 15]","[89, 35]",[0],"[1, 12]",없음,없음,없음,없음


In [289]:
type(game_major2[1][9999])

str

In [282]:
game_major2[1][1] == str(game_major2[1][1])

True

In [285]:
len(game_major2)

10000

In [317]:
for i in range(11):
    for j in range(10000):
        if (type(game_major2[i][j]) == list):
            game_major2[i][j] = str(game_major2[i][j])

In [318]:
for i in range(11):
    for j in range(10000):
        if (game_major2[i][j] is not None):
            game_major2[i][j] = ((game_major2[i][j][1:-1]))

In [335]:
for i in range(11):
    for j in range(10000):
        if (game_major2[i][j] is  None):
            game_major2[i][j] = [100]

In [336]:
game_major2

,0,1,2,3,4,5,6,7,8,9,10
0,[12],[46],"[37, 45]","[25, 66, 13]",[0],[0],[0],"[99, 47, 55]",[100],[100],[100]
1,[0],[0],[0],"[56, 13, 29]",[0],"[36, 6, 27]",[0],"[37, 34]",[17],[100],[100]
2,[0],[0],[0],[36],"[13, 57, 69]",[0],"[33, 69]",[48],[100],[100],[100]
3,[0],"[25, 66]",[0],"[14, 3]",[0],"[11, 23, 29]",[0],[57],[100],[100],[100]
4,[0],"[44, 36]",[0],"[16, 15, 19]","[67, 55]",[0],[0],[100],[100],[100],[100]
...,...,...,...,...,...,...,...,...,...,...,...
9995,[0],[0],"[99, 19, 22]","[56, 15, 25]","[22, 37, 23]",[0],[18],"[22, 24]",[100],[100],[100]
9996,[0],[0],"[78, 33]","[69, 27]","[46, 46, 15]",[0],"[47, 16]",[78],[100],[100],[100]
9997,[0],[67],[7],[0],[0],"[28, 16]","[13, 24, 24]",[0],"[57, 24]",[0],[100]
9998,[0],[0],"[37, 49, 1]","[69, 15]","[89, 35]",[0],"[1, 12]",[100],[100],[100],[100]


In [339]:
game_major2

,0,1,2,3,4,5,6,7,8,9,10
0,[12],[46],"[37, 45]","[25, 66, 13]",[0],[0],[0],"[99, 47, 55]",[100],[100],[100]
1,[0],[0],[0],"[56, 13, 29]",[0],"[36, 6, 27]",[0],"[37, 34]",[17],[100],[100]
2,[0],[0],[0],[36],"[13, 57, 69]",[0],"[33, 69]",[48],[100],[100],[100]
3,[0],"[25, 66]",[0],"[14, 3]",[0],"[11, 23, 29]",[0],[57],[100],[100],[100]
4,[0],"[44, 36]",[0],"[16, 15, 19]","[67, 55]",[0],[0],[100],[100],[100],[100]
...,...,...,...,...,...,...,...,...,...,...,...
9995,[0],[0],"[99, 19, 22]","[56, 15, 25]","[22, 37, 23]",[0],[18],"[22, 24]",[100],[100],[100]
9996,[0],[0],"[78, 33]","[69, 27]","[46, 46, 15]",[0],"[47, 16]",[78],[100],[100],[100]
9997,[0],[67],[7],[0],[0],"[28, 16]","[13, 24, 24]",[0],"[57, 24]",[0],[100]
9998,[0],[0],"[37, 49, 1]","[69, 15]","[89, 35]",[0],"[1, 12]",[100],[100],[100],[100]


In [345]:
game_major = game_major.rename(columns={0: 'FirstC', 1: 'SecondC',2: 'ThirdC', 3: 'FourC',4: 'FIveC', 5: 'SixC',6: 'SevenC', 7: 'EightC',8: 'NineC', 9: 'TenC',10: 'ElevenC'})

In [346]:
game_major

,FirstC,SecondC,ThirdC,FourC,FIveC,SixC,SevenC,EightC,NineC,TenC,ElevenC
0,3JarvanIV,3Sona,3Rakan,3XinZhao,2Neeko,2Karma,1Soraka,2Lulu,NotChampion,NotChampion,NotChampion
1,2Malphite,2Blitzcrank,1Vi,2Jinx,2Soraka,2ChoGath,2Kayle,1MissFortune,2Lulu,NotChampion,NotChampion
2,2Malphite,2Blitzcrank,2Ezreal,2Vi,1Jinx,2ChoGath,2MissFortune,2AurelionSol,NotChampion,NotChampion,NotChampion
3,2Leona,2Mordekaiser,2Jayce,2Ashe,2WuKong,2Jhin,2Thresh,2Ekko,NotChampion,NotChampion,NotChampion
4,2KhaZix,3KaiSa,3Annie,2Shaco,3Rumble,1Fizz,1MissFortune,NotChampion,NotChampion,NotChampion,NotChampion
...,...,...,...,...,...,...,...,...,...,...,...
9995,2Shen,1Rakan,3Ezreal,2Kassadin,3Kayle,2WuKong,1MissFortune,1Thresh,NotChampion,NotChampion,NotChampion
9996,2Poppy,2Zoe,3Ahri,3Neeko,3Syndra,2Lux,2Soraka,1Lulu,NotChampion,NotChampion,NotChampion
9997,1TwistedFate,2Blitzcrank,2Shen,2Ezreal,2Kassadin,2Irelia,2Kayle,2WuKong,1MissFortune,1Thresh,NotChampion
9998,2JarvanIV,2Sona,2Rakan,2XinZhao,2Neeko,2Karma,2Soraka,NotChampion,NotChampion,NotChampion,NotChampion


In [349]:
game_major2 = game_major2.rename(columns={0: 'FirstT', 1: 'SecondT',2: 'ThirdT', 3: 'FourT',4: 'FIveT', 5: 'SixT',6: 'SevenT', 7: 'EightT',8: 'NineT', 9: 'TenT',10: 'ElevenT'})

In [350]:
game_major2

,FirstT,SecondT,ThirdT,FourT,FIveT,SixT,SevenT,EightT,NineT,TenT,ElevenT
0,[12],[46],"[37, 45]","[25, 66, 13]",[0],[0],[0],"[99, 47, 55]",[100],[100],[100]
1,[0],[0],[0],"[56, 13, 29]",[0],"[36, 6, 27]",[0],"[37, 34]",[17],[100],[100]
2,[0],[0],[0],[36],"[13, 57, 69]",[0],"[33, 69]",[48],[100],[100],[100]
3,[0],"[25, 66]",[0],"[14, 3]",[0],"[11, 23, 29]",[0],[57],[100],[100],[100]
4,[0],"[44, 36]",[0],"[16, 15, 19]","[67, 55]",[0],[0],[100],[100],[100],[100]
...,...,...,...,...,...,...,...,...,...,...,...
9995,[0],[0],"[99, 19, 22]","[56, 15, 25]","[22, 37, 23]",[0],[18],"[22, 24]",[100],[100],[100]
9996,[0],[0],"[78, 33]","[69, 27]","[46, 46, 15]",[0],"[47, 16]",[78],[100],[100],[100]
9997,[0],[67],[7],[0],[0],"[28, 16]","[13, 24, 24]",[0],"[57, 24]",[0],[100]
9998,[0],[0],"[37, 49, 1]","[69, 15]","[89, 35]",[0],"[1, 12]",[100],[100],[100],[100]


In [358]:
game_major

,FirstC,SecondC,ThirdC,FourC,FIveC,SixC,SevenC,EightC,NineC,TenC,ElevenC
0,3JarvanIV,3Sona,3Rakan,3XinZhao,2Neeko,2Karma,1Soraka,2Lulu,NotChampion,NotChampion,NotChampion
1,2Malphite,2Blitzcrank,1Vi,2Jinx,2Soraka,2ChoGath,2Kayle,1MissFortune,2Lulu,NotChampion,NotChampion
2,2Malphite,2Blitzcrank,2Ezreal,2Vi,1Jinx,2ChoGath,2MissFortune,2AurelionSol,NotChampion,NotChampion,NotChampion
3,2Leona,2Mordekaiser,2Jayce,2Ashe,2WuKong,2Jhin,2Thresh,2Ekko,NotChampion,NotChampion,NotChampion
4,2KhaZix,3KaiSa,3Annie,2Shaco,3Rumble,1Fizz,1MissFortune,NotChampion,NotChampion,NotChampion,NotChampion
...,...,...,...,...,...,...,...,...,...,...,...
9995,2Shen,1Rakan,3Ezreal,2Kassadin,3Kayle,2WuKong,1MissFortune,1Thresh,NotChampion,NotChampion,NotChampion
9996,2Poppy,2Zoe,3Ahri,3Neeko,3Syndra,2Lux,2Soraka,1Lulu,NotChampion,NotChampion,NotChampion
9997,1TwistedFate,2Blitzcrank,2Shen,2Ezreal,2Kassadin,2Irelia,2Kayle,2WuKong,1MissFortune,1Thresh,NotChampion
9998,2JarvanIV,2Sona,2Rakan,2XinZhao,2Neeko,2Karma,2Soraka,NotChampion,NotChampion,NotChampion,NotChampion
